# 1. Azure OpenAI - 작업 진행 상황 표시

이 노트북에서는 **Azure OpenAI Service**를 사용하여 AI 에이전트의 작업 진행 상황을 실시간으로 표시하는 방법을 보여줍니다.

## 주요 내용
- 🔄 **스트리밍 응답**: 토큰 단위로 실시간 응답 표시
- 📊 **진행 메트릭**: 토큰/초, 경과 시간, 총 토큰 수
- 🔧 **도구 호출(Tool Calling)**: 함수 실행 과정 실시간 표시
- 📋 **단계별 표시**: "분석 중...", "도구 실행 중...", "응답 생성 중..."

---

## 1.1 환경 설정 및 라이브러리 임포트

In [1]:
import os
import time
import json
from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets

# Azure OpenAI SDK
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 로드 완료!")

✅ 라이브러리 로드 완료!


In [2]:
# Azure OpenAI 클라이언트 설정
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2025-01-01-preview")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o")

print("🔄 Azure OpenAI 클라이언트 설정 중...")

print(f"  - AZURE_OPENAI_ENDPOINT: {AZURE_OPENAI_ENDPOINT}")
print(f"  - AZURE_OPENAI_API_VERSION: {AZURE_OPENAI_API_VERSION}")
print(f"  - AZURE_OPENAI_DEPLOYMENT_NAME: {AZURE_OPENAI_DEPLOYMENT_NAME}")

if AZURE_OPENAI_ENDPOINT:
    token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

    client = AzureOpenAI(
        api_version=AZURE_OPENAI_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        azure_ad_token_provider=token_provider
    )
else:
    print("⚠️ Azure OpenAI 설정이 필요합니다.")
    print("  .env 파일에 다음 값을 설정해주세요:")
    print("  - AZURE_ENDPOINT (필수)")
    print("  - AZURE_API_KEY (선택 - 없으면 Azure AD 인증 사용)")
    print("  - AZURE_DEPLOYMENT_NAME (선택)")

🔄 Azure OpenAI 클라이언트 설정 중...
  - AZURE_OPENAI_ENDPOINT: https://ms-foundry-1023.cognitiveservices.azure.com/
  - AZURE_OPENAI_API_VERSION: 2024-12-01-preview
  - AZURE_OPENAI_DEPLOYMENT_NAME: gpt-4o


## 1.2 기본 스트리밍 - 토큰 단위 실시간 표시

Azure OpenAI API의 `stream=True` 옵션을 사용하면 응답이 생성되는 대로 토큰 단위로 받아볼 수 있습니다.
사용자에게 **AI가 현재 답변을 생성 중**임을 실시간으로 보여줍니다.

In [3]:
def basic_streaming_chat(text: str):
    """기본 스트리밍 채팅 - 토큰 단위로 실시간 표시"""
    
    print("🤖 Azure OpenAI 응답 생성 중...\n")
    print("-" * 50)
    
    # 스트리밍 응답 요청
    messages = [{"role": "user", "content": text}]
    stream = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=messages,
        stream=True
    )
    
    full_response = ""
    token_count = 0
    
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_response += content
            token_count += 1
            
            # 실시간으로 토큰 출력 (줄바꿈 없이)
            print(content, end="", flush=True)
    
    print("\n" + "-" * 50)
    print(f"\n✅ 완료! (약 {token_count} 토큰)")
    
    return full_response

# 테스트
response = basic_streaming_chat("Python의 장점을 3가지만 간단히 설명해주세요.")

🤖 Azure OpenAI 응답 생성 중...

--------------------------------------------------
1. **읽기 쉽고 간결한 문법**  
   Python은 직관적이고 간결한 문법을 통해 코드 가독성이 뛰어나며, 초보자도 쉽게 배울 수 있습니다.

2. **광범위한 라이브러리와 생태계**  
   머신러닝, 웹 개발, 데이터 분석 등 다양한 용도로 사용할 수 있는 방대한 표준 라이브러리와 오픈소스 생태계를 제공합니다.

3. **다양한 플랫폼과의 호환성**  
   운영 체제에 구애받지 않고 대부분의 플랫폼에서 동작하며, 코드 재사용 및 확장성이 뛰어납니다.
--------------------------------------------------

✅ 완료! (약 141 토큰)


## 1.3 향상된 진행 표시 - 메트릭과 함께

더 풍부한 사용자 경험을 위해 다음 정보를 함께 표시합니다:
- ⏱️ **경과 시간**: 응답 생성에 걸린 시간
- 📊 **토큰 수**: 현재까지 생성된 토큰 수
- 🚀 **토큰/초**: 초당 토큰 생성 속도

In [4]:
def streaming_with_metrics(prompt: str):
    """메트릭과 함께 스트리밍 표시"""
    
    start_time = time.time()
    
    stream = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[{"role": "user", "content": prompt}],
        stream=True
    )
    
    full_response = ""
    token_count = 0
    
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_response += content
            token_count += 1
            
            # 메트릭 계산
            elapsed = time.time() - start_time
            tokens_per_sec = token_count / elapsed if elapsed > 0 else 0
            
            # 화면 갱신
            clear_output(wait=True)
            
            # 상태 표시 (다크/라이트 테마 모두 지원)
            status_html = f"""
            <div style="font-family: monospace; padding: 12px; background: rgba(33, 150, 243, 0.15); border: 1px solid rgba(33, 150, 243, 0.4); border-radius: 8px; margin-bottom: 10px;">
                <div style="color: #64b5f6; font-weight: bold;">☁️ Azure OpenAI 응답 생성 중...</div>
                <div style="margin-top: 8px; color: #e0e0e0;">
                    ⏱️ 경과 시간: <b style="color: #fff;">{elapsed:.1f}초</b> | 
                    📊 토큰: <b style="color: #fff;">{token_count}</b> | 
                    🚀 속도: <b style="color: #fff;">{tokens_per_sec:.1f} 토큰/초</b>
                </div>
            </div>
            """
            display(HTML(status_html))
            
            # 현재까지의 응답 표시
            display(Markdown(full_response))
    
    # 완료 상태
    total_time = time.time() - start_time
    clear_output(wait=True)
    
    completion_html = f"""
    <div style="font-family: monospace; padding: 12px; background: rgba(76, 175, 80, 0.15); border: 1px solid rgba(76, 175, 80, 0.4); border-radius: 8px; margin-bottom: 10px;">
        <div style="color: #81c784; font-weight: bold;">✅ Azure OpenAI 응답 완료!</div>
        <div style="margin-top: 8px; color: #e0e0e0;">
            ⏱️ 총 시간: <b style="color: #fff;">{total_time:.2f}초</b> | 
            📊 총 토큰: <b style="color: #fff;">{token_count}</b> | 
            🚀 평균 속도: <b style="color: #fff;">{token_count/total_time:.1f} 토큰/초</b>
        </div>
    </div>
    """
    display(HTML(completion_html))
    display(Markdown(full_response))
    
    return full_response

# 테스트
response = streaming_with_metrics("머신러닝과 딥러닝의 차이점을 설명해주세요.")

머신러닝(Machine Learning)과 딥러닝(Deep Learning)은 모두 인공지능(AI)의 하위 분야로, 데이터를 이용해 학습하고 예측을 수행하는 기술입니다. 그러나 이 두 가지는 접근 방식과 동작 구조에서 차이가 있습니다. 아래에서 각각의 차이점을 설명드리겠습니다.

---

### 1. **정의와 개념**
- **머신러닝(Machine Learning)**  
  머신러닝은 데이터를 기반으로 한 알고리즘이 정해진 작업을 수행하도록 컴퓨터를 학습시키는 기술입니다. 머신러닝 모델은 사람이 직접 설계한 특징(feature)을 사용해 학습하고 결과를 도출합니다.
  
  예: 선형 회귀, 로지스틱 회귀, 의사결정나무, 랜덤 포레스트, 서포트 벡터 머신(SVM) 등

- **딥러닝(Deep Learning)**  
  딥러닝은 머신러닝의 하위 분야로, 인공신경망(Artificial Neural Network, ANN)을 기반으로 한 학습 기법입니다. 데이터를 통해 스스로 특징을 학습하며, 다층 구조(Deep Layer)를 활용해 복잡한 문제를 해결합니다. 인간의 뇌 신경구조를 모방했다는 점이 특징입니다.

  예: 합성곱 신경망(CNN), 순환 신경망(RNN), 트랜스포머(Transformer), 생성적 적대 신경망(GAN) 등

---

### 2. **특징 추출(Feature Extraction)의 차이**
- **머신러닝:**  
  머신러닝에서는 사람이 도메인 지식을 이용해 특징(feature)을 설계해야 합니다. 즉, 모델에 입력하기 전에 입력 데이터에서 의미 있는 특징을 추출하는 과정이 필요합니다.
  
  > 예: 이미지 인식에서는 경계선이나 색상 히스토그램 등을 사람이 설계해야 함.

- **딥러닝:**  
  딥러닝은 데이터로부터 중요한 특징을 자동으로 학습합니다. 여러 층의 신경망이 데이터를 분석하고 점점 더 고수준의 특징을 추출하는 방식을 사용합니다.

  > 예: 딥러닝 기반 이미지 인식에서는 원본 이미지를 그대로 입력하면 신경망이 스스로 경계선, 질감 등의 특징을 학습함.

---

### 3. **데이터 요구량**
- **머신러닝:**  
  상대적으로 적은 양의 데이터로도 모델을 학습할 수 있습니다. 작은 규모의 데이터셋이나 구조화된 데이터를 사용하는데 적합합니다.
  
  > 예: 엑셀 데이터나 정형화된 CSV 파일.

- **딥러닝:**  
  대규모 데이터셋을 필요로 합니다. 신경망이 특징을 스스로 학습하기 위해서는 수많은 데이터가 필요하며, 이를 처리하려면 고성능 컴퓨팅 자원도 요구됩니다.

  > 예: 이미지넷(ImageNet) 등 대량의 이미지를 활용한 학습.

---

### 4. **계산 자원**
- **머신러닝:**  
  계산 자원이 비교적 적게 필요합니다. CPU와 같은 표준 컴퓨팅 장비로 구현 가능하며, 학습 시간이 상대적으로 빠릅니다.

- **딥러닝:**  
  많은 계산 자원이 필요합니다. GPU 또는 TPU와 같은 고성능 병렬 컴퓨팅 하드웨어가 필수적이며, 학습 시간도 오래 걸리는 경우가 많습니다.

---

### 5. **복잡한 문제 해결 능력**
- **머신러닝:**  
  복잡도가 낮거나 중간 수준의 문제에 적합하며, 결과가 해석하기 쉬운 경우가 많습니다.  
  > 예: 금융 데이터 분석, 간단한 분류 문제 등

- **딥러닝:**  
  비구조화된 데이터(이미지, 음성, 텍스트 등)와 높은 복잡도를 가진 문제를 해결하는 데 뛰어납니다.  
  > 예: 객체 인식, 음성 인식, 자연어 처리 등

---

### 6. **응용 분야**
- **머신러닝:**  
  - 추천 시스템(추천 상품, 영화 추천 등)  
  - 금융 사기 탐지  
  - 간단한 의료 데이터 분석

- **딥러닝:**  
  - 음성 인식(AI 스피커, 번역기)  
  - 자율 주행(컴퓨터 비전)  
  - 딥페이크 생성  
  - 챗봇 및 질의응답 시스템(예: OpenAI의 GPT)

---

### 요약
| 구분         | 머신러닝                               | 딥러닝                          |
|--------------|----------------------------------------|---------------------------------|
| 학습 방식    | 사람이 직접 특징 설계                 | 신경망이 자동으로 특징 학습     |
| 학습 구조    | 단순 or 얕은 모델                     | 깊은 신경망 구조               |
| 데이터 요구량| 적은 데이터로 가능                   | 대량의 데이터 필요              |
| 계산 자원    | CPU로 충분                            | GPU/TPU 필요                   |
| 사용 사례    | 정형 데이터 분석                      | 비정형 데이터 분석              |

결론적으로 딥러닝은 머신러닝 기술을 더 확장한 고도화된 방법론으로 볼 수 있으며, 복잡하고 방대한 데이터를 다루는 데 더 적합합니다.

## 1.4 도구 호출(Tool Calling) - 기본 설정

Azure OpenAI의 Function Calling 기능을 사용하여 AI가 도구를 호출하는 과정을 표시합니다.
**"웹 검색 중..."**, **"계산 중..."** 등의 작업 표시가 가능합니다.

In [5]:
# 사용 가능한 도구 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "특정 도시의 현재 날씨 정보를 가져옵니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string", 
                        "description": "도시 이름 (예: 서울, 부산, 뉴욕)"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "온도 단위"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "수학 계산을 수행합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string", 
                        "description": "계산할 수학 표현식 (예: 15 + 27 * 3)"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "웹에서 정보를 검색합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string", 
                        "description": "검색할 쿼리"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

print("✅ 도구 정의 완료!")
print("  - get_weather: 날씨 조회")
print("  - calculate: 수학 계산")
print("  - search_web: 웹 검색")

✅ 도구 정의 완료!
  - get_weather: 날씨 조회
  - calculate: 수학 계산
  - search_web: 웹 검색


In [6]:
# 도구 실행 함수들 (실제 구현 또는 시뮬레이션)
def execute_tool(name: str, arguments: dict) -> str:
    """도구 실행 및 결과 반환"""
    
    if name == "get_weather":
        location = arguments.get("location", "Unknown")
        unit = arguments.get("unit", "celsius")
        # 실제로는 날씨 API 호출
        return json.dumps({
            "location": location,
            "temperature": 22 if unit == "celsius" else 72,
            "unit": unit,
            "condition": "맑음",
            "humidity": 45,
            "description": f"{location}의 현재 날씨는 맑고, 기온은 22°C입니다."
        }, ensure_ascii=False)
    
    elif name == "calculate":
        expression = arguments.get("expression", "0")
        try:
            # 안전한 수학 계산 (eval 대신 실제 구현 권장)
            result = eval(expression)
            return json.dumps({
                "expression": expression,
                "result": result,
                "description": f"{expression} = {result}"
            }, ensure_ascii=False)
        except Exception as e:
            return json.dumps({"error": str(e)}, ensure_ascii=False)
    
    elif name == "search_web":
        query = arguments.get("query", "")
        # 실제로는 검색 API 호출
        return json.dumps({
            "query": query,
            "results": [
                {"title": f"{query} 관련 정보 1", "snippet": "첫 번째 검색 결과입니다..."},
                {"title": f"{query} 관련 정보 2", "snippet": "두 번째 검색 결과입니다..."},
            ],
            "description": f"'{query}'에 대한 2개의 검색 결과를 찾았습니다."
        }, ensure_ascii=False)
    
    return json.dumps({"error": "Unknown tool"}, ensure_ascii=False)


# 도구 이름에 따른 아이콘 및 설명
TOOL_INFO = {
    "get_weather": {"icon": "🌤️", "label": "날씨 조회"},
    "calculate": {"icon": "🧮", "label": "계산 수행"},
    "search_web": {"icon": "🔍", "label": "웹 검색"}
}

print("✅ 도구 실행 함수 정의 완료!")

✅ 도구 실행 함수 정의 완료!


## 1.6 도구 호출 with 진행 상황 표시

AI가 도구를 호출할 때마다 **어떤 도구를 사용하는지** 실시간으로 표시합니다.
OpenAI나 Gemini에서 볼 수 있는 "웹 검색 중...", "계산 중..." 등의 표시와 유사합니다.

In [ ]:
def display_progress(steps: list, current_step: str = None, is_complete: bool = False):
    """진행 상황 표시 헬퍼 함수 (다크/라이트 테마 모두 지원)"""
    
    if is_complete:
        bg_color = "rgba(76, 175, 80, 0.15)"
        border_color = "rgba(76, 175, 80, 0.4)"
        header_color = "#81c784"
        header = "✅ 모든 작업 완료!"
    else:
        bg_color = "rgba(33, 150, 243, 0.15)"
        border_color = "rgba(33, 150, 243, 0.4)"
        header_color = "#64b5f6"
        header = "☁️ Azure OpenAI 처리 중..."
    
    html = f"""
    <div style="font-family: 'Segoe UI', sans-serif; padding: 15px; background: {bg_color}; border: 1px solid {border_color}; border-radius: 10px; max-width: 600px; margin-bottom: 10px;">
        <div style="font-weight: bold; color: {header_color}; margin-bottom: 10px;">{header}</div>
        <div style="border-left: 3px solid #0078d4; padding-left: 10px;">
    """
    
    for i, step in enumerate(steps):
        if step.get("status") == "completed":
            icon = "✅"
            color = "#81c784"  # 밝은 녹색
        elif step.get("status") == "running":
            icon = "⏳"
            color = "#64b5f6"  # 밝은 파란색
        else:
            icon = "⚪"
            color = "#9e9e9e"
        
        html += f"""
        <div style="padding: 5px 0; color: {color};">
            {icon} {step.get("text", "")}
            {f'<span style="color: #bdbdbd; font-size: 0.85em;"> ({step.get("detail", "")})</span>' if step.get("detail") else ""}
        </div>
        """
    
    html += "</div></div>"
    display(HTML(html))


def streaming_with_tools(prompt: str):
    """도구 호출과 함께 진행 상황 표시"""
    
    messages = [{"role": "user", "content": prompt}]
    steps = []
    all_tool_results = []
    
    iteration = 0
    max_iterations = 10  # 무한 루프 방지
    
    while iteration < max_iterations:
        iteration += 1
        
        # Azure OpenAI 호출
        steps.append({"text": "Azure OpenAI 호출 중...", "status": "running"})
        clear_output(wait=True)
        display_progress(steps)
        
        response = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT_NAME,
            messages=messages,
            tools=tools,
            stream=True
        )
        
        steps[-1]["status"] = "completed"
        steps[-1]["text"] = "Azure OpenAI 응답 수신"
        
        # 스트리밍 응답 처리
        full_content = ""
        tool_calls_data = []
        
        for chunk in response:
            if not chunk.choices:
                continue
                
            delta = chunk.choices[0].delta
            
            # 텍스트 응답
            if delta.content:
                full_content += delta.content
            
            # 도구 호출 수집
            if delta.tool_calls:
                for tc in delta.tool_calls:
                    # 새 도구 호출 시작
                    if tc.index is not None:
                        while len(tool_calls_data) <= tc.index:
                            tool_calls_data.append({"id": "", "name": "", "arguments": ""})
                        
                        if tc.id:
                            tool_calls_data[tc.index]["id"] = tc.id
                        if tc.function:
                            if tc.function.name:
                                tool_calls_data[tc.index]["name"] = tc.function.name
                            if tc.function.arguments:
                                tool_calls_data[tc.index]["arguments"] += tc.function.arguments
        
        # 도구 호출이 있는 경우
        if tool_calls_data:
            # 도구 호출 처리
            tool_calls_for_message = []
            
            for tc in tool_calls_data:
                tool_name = tc["name"]
                tool_info = TOOL_INFO.get(tool_name, {"icon": "🔧", "label": tool_name})
                
                # 진행 표시
                steps.append({
                    "text": f"{tool_info['icon']} {tool_info['label']} 실행 중...",
                    "status": "running"
                })
                clear_output(wait=True)
                display_progress(steps)
                
                # 도구 실행
                try:
                    args = json.loads(tc["arguments"]) if tc["arguments"] else {}
                except:
                    args = {}
                
                result = execute_tool(tool_name, args)
                
                # 결과 표시
                result_preview = json.loads(result).get("description", result[:50])
                steps[-1]["status"] = "completed"
                steps[-1]["text"] = f"{tool_info['icon']} {tool_info['label']} 완료"
                steps[-1]["detail"] = result_preview
                
                clear_output(wait=True)
                display_progress(steps)
                
                # 메시지에 추가
                tool_calls_for_message.append({
                    "id": tc["id"],
                    "type": "function",
                    "function": {"name": tool_name, "arguments": tc["arguments"]}
                })
                
                all_tool_results.append({
                    "tool_call_id": tc["id"],
                    "role": "tool",
                    "content": result
                })
            
            # Assistant 메시지 추가
            messages.append({
                "role": "assistant",
                "tool_calls": tool_calls_for_message
            })
            
            # 도구 결과 메시지 추가
            for tr in all_tool_results[-len(tool_calls_data):]:
                messages.append(tr)
            
            # 다음 반복
            continue
        
        # 최종 응답 (도구 호출 없음)
        if full_content:
            steps.append({"text": "최종 응답 생성 완료", "status": "completed"})
            clear_output(wait=True)
            display_progress(steps, is_complete=True)
            display(Markdown(full_content))
            return full_content
        
        break
    
    return None


# 테스트: 도구 호출이 필요한 질문
print("도구 호출 테스트 준비 완료!")
print("아래 셀에서 streaming_with_tools() 함수를 실행하세요.")

도구 호출 테스트 준비 완료!
아래 셀에서 streaming_with_tools() 함수를 실행하세요.


In [11]:
# 테스트 1: 날씨 + 계산
response = streaming_with_tools("서울의 현재 날씨와 (15 + 27) * 3의 계산 결과를 알려주세요.")

In [12]:
# 테스트 2: 웹 검색
response = streaming_with_tools("최신 AI 트렌드에 대해 검색해서 알려주세요.")

## 1.6 스트리밍 응답 구조 이해하기

> 📖 참고: [Azure OpenAI REST API Reference - createChatCompletionStreamResponse](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/reference?view=foundry-classic#createchatcompletionstreamresponse)

스트리밍 모드(`stream=True`)에서는 **`createChatCompletionStreamResponse`** 형태로 응답이 옵니다.
이 노트북 코드에서 사용한 필드들을 API 문서 기준으로 설명합니다.

---

### 📦 전체 응답 구조 (chunk 단위)

```python
for chunk in response:  # 각 chunk는 createChatCompletionStreamResponse 객체
    chunk.id                    # 고유 식별자 (모든 청크가 동일)
    chunk.choices               # 응답 선택지 배열
    chunk.created               # 생성 타임스탬프
    chunk.model                 # 사용된 모델명
    chunk.object                # 항상 "chat.completion.chunk"
```

---

### 🎯 코드에서 사용한 필드 매핑

| 코드 | API 필드 | 설명 |
|------|----------|------|
| `chunk.choices` | `choices` | 응답 선택지 배열 (보통 1개) |
| `chunk.choices[0].delta` | `chatCompletionStreamResponseDelta` | **스트리밍용 델타 객체** (일반 응답의 `message` 대신 사용) |
| `delta.content` | `delta.content` | 이번 청크에서 생성된 텍스트 조각 |
| `delta.tool_calls` | `delta.tool_calls` | 도구 호출 정보 (점진적으로 수신) |

---

### 🔄 delta vs message

**일반 응답** (`stream=False`):
```python
response.choices[0].message.content      # 전체 텍스트
response.choices[0].message.tool_calls   # 전체 도구 호출 목록
```

**스트리밍 응답** (`stream=True`):
```python
chunk.choices[0].delta.content           # 토큰 조각 (누적 필요)
chunk.choices[0].delta.tool_calls        # 도구 호출 조각 (누적 필요)
```

`delta`는 "변화량"을 의미합니다. 매 청크마다 **새로 생성된 부분만** 전달되므로 코드에서 누적해야 합니다.

---

### 🔧 Tool Calls 스트리밍 처리 (코드 해설)

```python
# delta.tool_calls 구조: chatCompletionMessageToolCallChunk
for tc in delta.tool_calls:
    tc.index        # 도구 호출 인덱스 (여러 도구 호출 시 구분)
    tc.id           # 도구 호출 고유 ID (첫 청크에서만 수신)
    tc.type         # "function" (항상 고정)
    tc.function.name       # 함수 이름 (첫 청크에서만 수신)
    tc.function.arguments  # 인자 JSON 문자열 (청크 단위로 분할 수신)
```

**코드에서의 처리:**
```python
# 인덱스별로 데이터 누적
tool_calls_data[tc.index]["id"] = tc.id
tool_calls_data[tc.index]["name"] = tc.function.name
tool_calls_data[tc.index]["arguments"] += tc.function.arguments  # 문자열 누적!
```

인자(`arguments`)는 `{"location": "서울"}` 같은 JSON이 `{"loc` → `ation":` → ` "서울"}` 처럼 조각나서 오므로 **반드시 누적**해야 합니다.

---

### 📋 finish_reason 값

| 값 | 의미 |
|----|------|
| `stop` | 정상 완료 |
| `tool_calls` | 도구 호출 필요 (다음 요청에 도구 결과 포함 필요) |
| `length` | max_tokens 도달 |
| `content_filter` | 콘텐츠 필터링 |

스트리밍에서는 마지막 청크에서만 `finish_reason`이 설정됩니다.

---

## 요약

이 노트북에서 다룬 Azure OpenAI 기반 작업 진행 표시 방법:

| 방식 | 설명 | 사용 사례 |
|------|------|-----------|
| **기본 스트리밍** | 토큰 단위 실시간 출력 | 간단한 채팅 인터페이스 |
| **메트릭 표시** | 시간/토큰/속도 정보 | 성능 모니터링 필요 시 |
| **단계별 표시** | 분석→구성→생성 단계 | 복잡한 작업 시각화 |
| **도구 호출** | 함수 실행 과정 표시 | 에이전트 작업 시각화 |

### 핵심 포인트
- `stream=True` 옵션으로 실시간 토큰 스트리밍
- `tools` 파라미터로 Function Calling 활성화
- 도구 호출 시 `tool_calls` 응답 처리
- `tool` 역할 메시지로 도구 결과 전달

**다음 노트북**: Microsoft Foundry Agent를 사용한 에이전트 단계별 표시 →